In [1]:
from datasets import load_dataset

ds = load_dataset("dominguesm/alpaca-data-pt-br")

In [2]:
from transformers import AutoTokenizer

# Example usage of the GPT model
tokenizer = AutoTokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")

In [3]:
df = ds["train"].to_pandas()

In [4]:
df["concat"] = (
    df["instruction"] + "\n\n" + df["input"].fillna("") + "\n\n" + df["output"]
)

In [5]:
# create a column with count of tokens for each row
df["tokens"] = df["concat"].apply(lambda x: len(tokenizer.encode(x)))

In [6]:
df["tokens"].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

count    51759.000000
mean        74.283371
std         40.517703
min         10.000000
25%         38.000000
50%         67.000000
75%        110.000000
90%        127.000000
95%        136.000000
99%        159.000000
max        838.000000
Name: tokens, dtype: float64

## Train

In [7]:
import torch

from model import GPT, GPTConfig

device = "cuda" if torch.cuda.is_available() else "cpu"

config = GPTConfig(
    block_size=128,
    vocab_size=tokenizer.vocab_size,
    n_layer=4,
    n_head=4,
    n_embd=128,
    n_experts=8,
    capacity_factor=1.25,
    k=2,
    experts_weight=0.01,
    router_weight=0.001,
    dropout=0.2,
    bias=True,
)

model = GPT(config)

number of parameters: 10.92M


In [8]:
inputs = tokenizer(
    df["concat"].tolist(),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=128,
)

input_ids = inputs["input_ids"]
targets = input_ids.clone()

targets[:, :-1] = input_ids[:, 1:]
targets[:, -1] = -1

# convert targets 0 to -1
targets[targets == 0] = -1

# print("input ids:", inputs["input_ids"], "shape:", inputs["input_ids"].shape)

In [9]:
# invert 0 -> 1 and 1 -> 0 in attention mask
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# create a new upper triangular mask
upper_mask = torch.triu(torch.ones(input_ids.shape[1], input_ids.shape[1]), diagonal=1)

# apply the mask to the attention mask
attention_mask = attention_mask.unsqueeze(1) * upper_mask.unsqueeze(0)

# conver to bool
attention_mask = attention_mask.bool()

In [10]:
model(input_ids[:5, :], attention_mask=attention_mask[:5, :, :])[0].shape

torch.Size([5, 1, 50257])

### Train loop

In [11]:
# send data to device
model = model.to(device)
input_ids = input_ids[:32, :].to(device)  # limit to 16 for testing
attention_mask = attention_mask[:32, :, :].to(device)
targets = targets[:32, :].to(device)

In [12]:
from tqdm.notebook import tqdm

with torch.no_grad():
    logits, loss = model(input_ids, attention_mask, targets)
    # print("logits:", logits, "shape:", logits.shape)
print("loss:", loss.item() if loss is not None else "N/A")
if loss is not None:
    perplexity = torch.exp(loss)
    print("perplexity:", perplexity.item())

# pred_tokens = logits.argmax(dim=-1)
# tokens = pred_tokens[0].tolist()
# print("predicted tokens:", tokens)
# print("predicted text:", tokenizer.decode(tokens))

optimizer = model.configure_optimizers(
    weight_decay=0.0,
    learning_rate=3e-3,
    betas=(0.9, 0.95),
    device_type="cuda",
)

pbar = tqdm(range(1000), desc="Training Epochs")
for epoch in pbar:
    model.train()
    optimizer.zero_grad()
    logits, loss = model(input_ids, attention_mask, targets)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        logits, loss = model(input_ids, attention_mask, targets)

    perplexity = torch.exp(loss)
    pbar.set_postfix(loss=loss.item(), perplexity=perplexity.item())

    pred_tokens = logits.argmax(dim=-1)
    tokens = pred_tokens[0].tolist()


loss: 10.92919921875
perplexity: 55781.59375
using fused AdamW: True


Training Epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids[:, :1].clone(),
        max_new_tokens=10,
        temperature=0.1,
        top_k=1,
        greedy=False,
    )
generated_text = tokenizer.decode(generated_ids[0].tolist())
_df = df.head(32)
_df["generated_text"] = [
    tokenizer.decode(generated_ids[i].tolist()) for i in range(len(generated_ids))
]
print(f"Generated text after epoch {epoch + 1}: {generated_text}")


Generated text after epoch 358: Dplplplplplplplplplpl


C:\Users\mathe\AppData\Local\Temp\ipykernel_4208\653658135.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['generated_text'] = [tokenizer.decode(generated_ids[i].tolist()) for i in range(len(generated_ids))]


In [15]:
_df.head(16)

,instruction,input,output,concat,tokens,generated_text
0,Dê três dicas para se manter saudável.,,1. Coma uma dieta equilibrada e certifique-se ...,Dê três dicas para se manter saudável.\n\n\n\n...,62,Dplplplplplplplplplpl
1,Quais são as três cores primárias?,,"As três cores primárias são vermelho, azul e a...",Quais são as três cores primárias?\n\n\n\nAs t...,23,Qu slo slo slo slo slo slo slo slo slo slo
2,Descreva a estrutura de um átomo.,,"Um átomo é composto de um núcleo, que contém p...",Descreva a estrutura de um átomo.\n\n\n\nUm át...,78,Descrecrecrecrecrecrecrecrecre esta
3,Como podemos reduzir a poluição do ar?,,Há várias maneiras de reduzir a poluição do ar...,Como podemos reduzir a poluição do ar?\n\n\n\n...,101,Comolhelhelhelhelhelhelhelhelhelhe
4,Finja que você é um gerente de projeto de uma ...,,Eu tive que tomar uma decisão difícil quando e...,Finja que você é um gerente de projeto de uma ...,135,Fin for for for for for for for for for for
5,Identifique o estranho.,"Twitter, Instagram, Telegrama",Telegrama,"Identifique o estranho.\n\nTwitter, Instagram,...",19,Identdentdentdentdentdentdentdentdentdent
6,Explicar por que a seguinte fração é equivalen...,4/16,A fração 4/16 é equivalente a 1/4 porque ambos...,Explicar por que a seguinte fração é equivalen...,62,Explicplicplicplicplicplicplicplicplicplic
7,Escreva uma história curta na narração em terc...,,John estava em uma encruzilhada em sua vida. E...,Escreva uma história curta na narração em terc...,133,Es 37 37 37 37 37 37 37 37 37 37
8,Avalie esta frase para erros ortográficos e gr...,Ele finished sua refeição e deixou o restourant,Ele terminou a refeição e saiu do restaurante.,Avalie esta frase para erros ortográficos e gr...,36,AAAAAAAAAAA
9,Como Júlio César morreu?,,Júlio César foi assassinado por um grupo de at...,Como Júlio César morreu?\n\n\n\nJúlio César fo...,50,Comolhelhelhelhelhelhelhelhelhelhe
